In [1]:
import pandas as pd
import numpy as np
import time
from rdkit import Chem
from rdkit.Chem import AllChem
from tdc.multi_pred import DDI
from sklearn.metrics import roc_auc_score,confusion_matrix,f1_score
from sklearn.ensemble import RandomForestClassifier

In [8]:
import pandas as pd


data1 = pd.read_csv("trainforstage1.csv")
data2 = pd.read_csv("testforstage1.csv")

data1 = data1.drop(["Unnamed: 0"],axis=1)
data1 = data1.rename(columns={'Drug1': 'smiles1','Drug2': 'smiles2',"Y" :'type','Drug1_ID':'d1','Drug2_ID':'d2'})

#data2 = data2.drop(['Drug1_ID', 'Drug2_ID',"Unnamed: 0"],axis=1)
data2 = data2.drop(["Unnamed: 0"],axis=1)
data2 = data2.rename(columns={'Drug1': 'smiles1','Drug2': 'smiles2',"Y" :'type','Drug1_ID':'d1','Drug2_ID':'d2'})

merged = pd.merge(data1,data2,on=['smiles1', 'smiles2','type','d1','d2'],how='outer')
#for i in range(len(merged)):
#    if merged.iloc[i,4] != 0:
#        merged.iloc[i,4] = 1
merged = merged.sample(frac=1).reset_index(drop=True)
mt = merged.iloc[:int(len(merged)*0.7),:]
mte = merged.iloc[int(len(merged)*0.7):int(len(merged)*0.85),:]
mv = merged.iloc[int(len(merged)*0.85):,:]       
#mt.to_csv('ddi_training.csv')
#mte.to_csv('ddi_test.csv')
#mv.to_csv('ddi_validation.csv')

172627.0


In [7]:
data = pd.read_csv('ddinter.csv')

drugtrans = pd.read_csv('drugbank_smile.csv')
transdic = {}
for i in range(len(drugtrans)):
    transdic[drugtrans.iloc[i,1]] = drugtrans.iloc[i,2]
dele = []
for i in range(len(data)):
    try:
        data.iloc[i,1] = transdic[data.iloc[i,1]]
        try:
            data.iloc[i,3] = transdic[data.iloc[i,3]]
        except:
            dele.append(i)
    except:
        dele.append(i)
    if data.iloc[i,4] == 'Unknown':
        dele.append(i)
dele = list(set(dele))

In [8]:
data = data.drop(dele)


In [12]:
data = data.rename(columns={"DDInterID_A":"Drug1_ID","DDInterID_B":"Drug2_ID"})


In [13]:
data

,Drug1_ID,Drug_A,Drug2_ID,Drug_B,Level
0,DDInter975,[Fe],DDInter582,[H][C@]12CN3C=C(C(=O)NCC4=CC=C(F)C=C4F)C(=O)C(...,Major
1,DDInter582,[H][C@]12CN3C=C(C(=O)NCC4=CC=C(F)C=C4F)C(=O)C(...,DDInter725,[Fe++].[H]\C(=C(\[H])C([O-])=O)C([O-])=O,Major
2,DDInter582,[H][C@]12CN3C=C(C(=O)NCC4=CC=C(F)C=C4F)C(=O)C(...,DDInter726,[Fe++].[H][C@@](O)(CO)[C@@]([H])(O)[C@]([H])(O...,Major
3,DDInter582,[H][C@]12CN3C=C(C(=O)NCC4=CC=C(F)C=C4F)C(=O)C(...,DDInter1114,[Mg++].[Cl-].[Cl-],Major
4,DDInter582,[H][C@]12CN3C=C(C(=O)NCC4=CC=C(F)C=C4F)C(=O)C(...,DDInter1115,[Mg++].[Mg++].[Mg++].OC(CC([O-])=O)(CC([O-])=O...,Major
...,...,...,...,...,...
10870,DDInter1903,NC(N)=O,DDInter1443,OC1=CC=C(C=C1)C1(OC(=O)C2=CC=CC=C12)C1=CC=C(O)...,Moderate
10872,DDInter1943,CC(C)CCC[C@@H](C)CCC[C@@H](C)CCC[C@]1(C)CCC2=C...,DDInter542,OC1=C(CC2=C(O)C3=C(OC2=O)C=CC=C3)C(=O)OC2=C1C=...,Moderate
10893,DDInter1144,N[C@@H](CC1=CC=C(C=C1)N(CCCl)CCCl)C(O)=O,DDInter1964,[Cl-].[Cl-].[Zn++],Minor
10894,DDInter1964,[Cl-].[Cl-].[Zn++],DDInter1145,CCOC(=O)[C@H](CC1=CC=C(F)C=C1)NC(=O)[C@@H](N)C...,Minor
